In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import sys
sys.path.append('..')
import time

import importlib

from astroLuSt.monitoring import (
    timers as almt,
    errorlogging as alme,
    formatting as almf
)




# ExecTimer

## Standard Usage

In [ ]:


importlib.reload(almt)

ET = almt.ExecTimer(verbose=1)

ET.checkpoint_start('Sup Task', comment='Global Task')

ET.checkpoint_start('Sub Task1')
time.sleep(2)
ET.checkpoint_end('Sub Task1')
ET.checkpoint_start('Sub Task2')
time.sleep(3)
ET.checkpoint_end('Sub Task2', comment='TEST')

ET.checkpoint_end('Sup Task')

ET.estimate_runtime('Sup Task', nrepeats=4, ndone=1)

ET.df_protocoll


## Usage to have an up-to-date estimate of runtimes

In [ ]:

#initialize the timer
ET = almt.ExecTimer(verbose=0)

repeats = 5 #number of times to repeat the loop


#Execute some loop
ET.checkpoint_start('Global Task')  #to monitor total (actual) duration
for idx in range(repeats):

    #monitor duration of each loop-execution
    ET.checkpoint_start(f'Subtask{idx}')

    #generate random sleeptime (should average out to be 1)
    time.sleep(np.random.normal(1, 0.5, size=1)[0])
    
    ET.checkpoint_end(f'Subtask{idx}')

    #estimate the runtime from the current point of view
    ET.estimate_runtime(r'Subtask', repeats, idx+1)

ET.checkpoint_end('Global Task')

ET.df_protocoll

## Usage as decorator

### Timing one function

In [ ]:
ET.verbose = 1
@ET.time_exec(start_kwargs={'comment':'timed_func'})
def timed_func():
    time.sleep(np.random.rand())
    return 'FINISHED'
tf = timed_func()


### Getting execution statistics

In [ ]:
ET.verbose = 0
@ET.get_execstats(n=10, metrics=None, drop_from_df_protocoll=True)
def evaluated_func():
    time.sleep(np.random.rand())
        
    return 'FINISHED'



ef = evaluated_func()

display(ef)

display(ET.df_protocoll)



# LogErrors

In [15]:
import astroLuSt.monitoring.errortesting as almet
from joblib import Parallel, delayed
importlib.reload(alme)

def subfunc(idx):
    n = np.random.choice(range(3), size=1)
    try:
        if n==0:
            almet.raise_concaterr(n)
        elif n == 1:
            almet.raise_div0()
        elif n == 2:
            almet.raise_nested_exception()
    except Exception as e:
        LE.print_exc(e, prefix='Pre', suffix=f'Suf{n}')
        df_le_temp = LE.exc2df(e, prefix='Pre', suffix=f'Suf{idx}', store=True)
            



x = pd.DataFrame(
    columns=['a', 'b'],
#      data=[[1,2],[2,3]]
)

LE = alme.LogErrors(verbose=0)


_ = Parallel(n_jobs=5, backend='threading')(delayed(subfunc)(idx=idx) for idx in range(5))
    

# print(LE.df_errorlog['exception'].iloc[0])
LE.df_errorlog


/home/lukas/github/astroLuSt/demos/../astroLuSt/monitoring/errorlogging.py:277: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_temp.index = [idx]*df_temp.shape[0]


,exception,prefix,suffix,file,line,problem_line,error_message,time,squeezed
0,"Traceback (most recent call last):\n File ""/h...",Pre,Suf1,/home/lukas/github/astroLuSt/demos/../astroLuS...,13,raise ValueError('custom error'),ValueError: custom error in except in except,2024-01-08 08:46:11.346035,False
0,"Traceback (most recent call last):\n File ""/h...",Pre,Suf1,/home/lukas/github/astroLuSt/demos/../astroLuS...,17,raise ValueError('custom error in except'),ValueError: custom error in except in except,2024-01-08 08:46:11.346035,False
0,"Traceback (most recent call last):\n File ""/h...",Pre,Suf1,/tmp/ipykernel_86853/1178136544.py,13,almet.raise_nested_exception(),ValueError: custom error in except in except,2024-01-08 08:46:11.346035,False
0,"Traceback (most recent call last):\n File ""/h...",Pre,Suf1,/home/lukas/github/astroLuSt/demos/../astroLuS...,19,raise ValueError('custom error in except in ex...,ValueError: custom error in except in except,2024-01-08 08:46:11.346035,False
1,"Traceback (most recent call last):\n File ""/t...",Pre,Suf2,/tmp/ipykernel_86853/1178136544.py,11,almet.raise_div0(),ZeroDivisionError: division by zero,2024-01-08 08:46:11.349025,False
1,"Traceback (most recent call last):\n File ""/t...",Pre,Suf2,/home/lukas/github/astroLuSt/demos/../astroLuS...,5,return 1/0,ZeroDivisionError: division by zero,2024-01-08 08:46:11.349025,False
2,"Traceback (most recent call last):\n File ""/h...",Pre,Suf3,/home/lukas/github/astroLuSt/demos/../astroLuS...,13,raise ValueError('custom error'),ValueError: custom error in except in except,2024-01-08 08:46:11.351537,False
2,"Traceback (most recent call last):\n File ""/h...",Pre,Suf3,/home/lukas/github/astroLuSt/demos/../astroLuS...,17,raise ValueError('custom error in except'),ValueError: custom error in except in except,2024-01-08 08:46:11.351537,False
2,"Traceback (most recent call last):\n File ""/h...",Pre,Suf3,/tmp/ipykernel_86853/1178136544.py,13,almet.raise_nested_exception(),ValueError: custom error in except in except,2024-01-08 08:46:11.351537,False
2,"Traceback (most recent call last):\n File ""/h...",Pre,Suf3,/home/lukas/github/astroLuSt/demos/../astroLuS...,19,raise ValueError('custom error in except in ex...,ValueError: custom error in except in except,2024-01-08 08:46:11.351537,False


# Formatted Printing

In [ ]:
importlib.reload(almf)
verbose = 3

for level in range(3):
    almf.printf(
        msg=f'info at level {level}',
        type='INFO',
        level=level,
        start=4*' ',
        verbose=verbose
    )